## Convert image to binary array, add padding find start and goal coordinates

In [99]:
# import the required packages
from PIL import Image
import numpy as np

# convert the maze image to grayscale
image = Image.open('test1.png').convert('L')
w, h = image.size # get the dimenssions

# resize every image to a fixed shape
#image = image.resize((100,100))

# set all white pixels to 1 and black to 0
binary = image.point(lambda p: p > 128 and True)  # 0 is blocked and 1 is unblocked

# Resize the binary so that we can reduce lot of 0's and 1's to traverse - this value is hardcoded for now
# we need to figure out a way to set w,h value without lossing information
# binary = binary.resize((500,800),Image.NEAREST)
w, h = binary.size
# converting image to numpy array
nim = np.array(binary)
# Padding for neighbors
nim= np.pad(nim, pad_width=5, mode='constant', constant_values=0)
# To identify which has the opening and closing
points = [nim[5],nim[h+4],nim[:,5],nim[:,w+4]]
# points = [nim[0],nim[h-1],nim[:,0],nim[:,w-1]]
res=[]
point =0
i=0
for i in range(len(points)):
    count=0
    if(len(res)<2):
        point= np.sum(points[i])
        if(point!=0):
            if(i<=1):
                for j in range(len(points[i])):
                    if(i==0):
                        k=5
                    if(i==1):
                        k=h+4
                    if(nim[k][j]==0):
                        continue
                    if(nim[k][j]==1):
                        count+=1
                        if(count==int(point/2)):
                            res.append((k,j))
                            break
            else:
                for j in range(len(points[i])):
                    if(i==2):
                        k=5
                    if(i==3):
                        k=w+4
                    if(nim[j][k]==0):
                        continue
                    if(nim[j][k]==1):
                        count+=1
                        if(count==point/2):
                            res.append((j,k))

    else:
        break
# Start and Stop
start=res[0]
goal=res[1]
    
# start=res[0]
# stop=res[1]
# print(start,stop)

# print(nim.shape)
# for r in range(1298,1300):
#     for c in range(w):
#         print(nim[r,c],end='')
#     print()
# for r in range(1298,1300):
#     for c in range(w):
#         if(nim[r,c]==1):
#             print(r,c)
# print(w,h)
# for r in range(h+10):
#     for c in range(w+10):
#         print(nim[r,c],end='')
#     print()

## Finding path

In [100]:
import sys
from PIL import Image
import numpy
import cv2

def AStar(start, goal, neighbor_nodes, distance, cost_estimate):
    def reconstruct_path(came_from, current_node):
        path = []
        while current_node is not None:
            path.append(current_node)
            current_node = came_from[current_node]
        return list(reversed(path))

    g_score = {start: 0}
    f_score = {start: g_score[start] + cost_estimate(start, goal)}
    openset = {start}
    closedset = set()
    came_from = {start: None}

    while openset:
        current = min(openset, key=lambda x: f_score[x])
        if current == goal:
            return reconstruct_path(came_from, goal)
        openset.remove(current)
        closedset.add(current)
        for neighbor in neighbor_nodes(current):
            if neighbor in closedset:
                continue
            if neighbor not in openset:
                openset.add(neighbor)
            tentative_g_score = g_score[current] + distance(current, neighbor)
            if tentative_g_score >= g_score.get(neighbor, float('inf')):
                continue
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = tentative_g_score + cost_estimate(neighbor, goal)
    return []


def is_blocked(p):
    x,y = p
    pixel = nim[x,y]
    if (pixel == 0):
        return True

def von_neumann_neighbors(p):
    x, y = p
    neighbors = [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]
    return [p for p in neighbors if not is_blocked(p)]

def manhattan(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

def squared_euclidean(p1, p2):
    return (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2

#3
# start = (400, 984)
# goal = (398, 25)

#2
# start = (872, 1296)
# goal = (638, 4)


# invoke: python mazesolver.py <mazefile> <outputfile>[.jpg|.png|etc.]

distance = squared_euclidean
heuristic = squared_euclidean

# 9 path is the solution path i dont know why there is another path in between should try different algo for path maybe.
path = AStar(start, goal, von_neumann_neighbors, distance, heuristic)
for position in path:
    x,y = position
    nim[x,y] = 9
result = numpy.ones([h+5,w+5,3])
for r in range(5,h+5):
    for c in range(5,w+5):
        i=1
        j=1
        if nim[r,c] == 0:
            result[r,c,:] = (0,0,0)
        elif nim[r,c] == 1:
            result[r,c,:] = (255,255,255)
        elif nim[r,c] == 9:
            result[r,c,:] = (255,0,0)
result = result[5:,5:]
# cnt = 0
# for r in range(h):
#     for c in range(w):
#         if(result[r,c] == (255,0,0)):
#             cnt = cnt+1
# print(cnt)
        #print(nim[r,c],end='')
# path_img.save(sys.argv[2])

In [101]:
# import cv2
# for r in range(5,h+4):
#     for c in range(5,w+4):
#         if nim[r,c] == 9:
#             result[r,c,:] = (0,255,0)
#     cv2.imshow("result",result)
#     k = cv2.waitKey(1) & 0xFF
#     if k == 27:
#         break
# cv2.destroyAllWindows()

In [102]:
import cv2
cv2.imwrite("result.jpg", result)


True

In [103]:
#             while(nim[r,c+i] == 1 and i<=5 and c+i<w):
#                 result[r,c,:] = (0,255,0)
#                 nim[r,c+i] =2
#                 i=i+1
#             while(nim[r,c-i] == 1 and 10-i>=0 and c-i>w):
#                 result[r,c-i,:] = (0,255,0)
#                 nim[r,c-i] = 2
#                 i=i+1
#             while(nim[r+j,c] == 1 and 10-j>=0 and c-j>h):
#                 result[r+j,c,:] = (0,255,0)
#                 nim[r+j,c] = 2
#                 j=j+1
#             while(nim[r-j,c] == 1 and 10-j>=0 and c-j>h):
#                 result[r-j,c,:] = (0,255,0)
#                 nim[r-j,c] = 2
#                 j=j+1